In [8]:
# Import Required Libraries
library(dplyr)
library(tsibble)
library(fable)
library(ggplot2)
library(tidyr)
library(purrr)
library(feasts)

# Source Helper Functions
source("baseline_helpers.R")
source("../Data_Inspection/data_cleaning_helpers.R")


In [9]:
# Clean validation data
validation <- get_validation_data()


## Load and Clean Train Data

In [10]:
# Load dates
dates <- read.csv("../calendar_afcs2025.csv") |> mutate(date = as.Date(date, format = "%m/%d/%Y"))
dates <- dates |> rename(day = date)

# added this later, might break everything, gotta check it
dates <- dates |> as_tsibble(index = day, key = wm_yr_wk)

# adding dummy variables for each weekday
dates <- dates |>
  mutate(
    wd_monday    = as.integer(weekday == "Monday"),
    wd_tuesday   = as.integer(weekday == "Tuesday"),
    wd_wednesday = as.integer(weekday == "Wednesday"),
    wd_thursday  = as.integer(weekday == "Thursday"),
    wd_friday    = as.integer(weekday == "Friday"),
    wd_saturday  = as.integer(weekday == "Saturday")
  )

dates <- dates |>
  mutate(
    m_january   = as.integer(month == 1),
    m_february  = as.integer(month == 2),
    m_march     = as.integer(month == 3),
    m_april     = as.integer(month == 4),
    m_may       = as.integer(month == 5),
    m_june      = as.integer(month == 6),
    m_july      = as.integer(month == 7),
    m_august    = as.integer(month == 8),
    m_september = as.integer(month == 9),
    m_october   = as.integer(month == 10),
    m_november  = as.integer(month == 11)
  )

# way too many unique event names so many we just focus on this,
# it's implemented with an or because event1 and event2 are always different type,
# and they can be in either order

# adding dummy variables for each event category
dates <- dates |>
  mutate(
    et_sporting  = as.integer((event_type_1 == "Sporting" | event_type_2 == "Sporting") %in% TRUE),
    et_cultural  = as.integer((event_type_1 == "Cultural" | event_type_2 == "Cultural") %in% TRUE),
    et_national  = as.integer((event_type_1 == "National" | event_type_2 == "National") %in% TRUE),
    et_religious = as.integer((event_type_1 == "Religious" | event_type_2 == "Religious") %in% TRUE)
  )


# event name

train <- get_train_data()


# take subset of train and validation

In [11]:
start <- 1
end <- 1000
products <- sprintf("FOODS_3_%03d", start:end)

train <- train |> filter(product %in% products)
validation <- validation |> filter(product %in% products)


## Baseline Prediction & Evaluation

Mean baseline

In [ ]:
means <- train |>
  as_tibble() |>
  group_by(product) |>
  summarise(mean_sales = mean(sales))

mean_forecasts <- new_data(ungroup(train), n = 28) |>
  left_join(means, by = "product") |>
  rename(sales = mean_sales) |>
  as_tibble()

aligned_data <- align_predictions(mean_forecasts, validation)
accuracy_mean <- calculate_metrics(aligned_data)
calculate_metrics(aligned_data)
calculate_metrics(aligned_data |> group_by(product)) |> arrange(-RMSE)
write.csv(calculate_metrics(aligned_data |> group_by(product)) |> arrange(-RMSE), "baseline_mean_accuracy.csv", row.names = FALSE)


RMSE,MAE
<dbl>,<dbl>
3.420624,1.639814


product,RMSE,MAE
<chr>,<dbl>,<dbl>
FOODS_3_090,26.841167,20.384568
FOODS_3_586,24.020672,20.414084
FOODS_3_555,21.122581,17.909473
FOODS_3_120,20.933727,17.935349
FOODS_3_804,19.112404,13.897954
FOODS_3_252,17.485972,14.816407
FOODS_3_444,16.553015,14.760851
FOODS_3_635,13.777313,13.777313
FOODS_3_808,13.686879,13.686879


In [13]:
meadians <- train |>
    as_tibble() |>
    group_by(product) |>
    summarise(median_sales = median(sales))

median_forecasts <- new_data(ungroup(train), n = 28) |>
    left_join(meadians, by = "product") |>
    rename(sales = median_sales) |>
    as_tibble()

aligned_data <- align_predictions(median_forecasts, validation)
accuracy_median <- calculate_metrics(aligned_data)
accuracy_median


RMSE,MAE
<dbl>,<dbl>
3.583065,1.587007


Naive Baseline

In [ ]:
path <- "models/baseline/naive.rds"

# load fit or fit it and store the fit
if (file.exists(path)) {
  fit <- readRDS(path)
} else {
  fit <- train |>
    model(
      NAIVE(sales)
    )
  saveRDS(fit, path)
}


naive_forecasts <- fit |>
  forecast(h = 28) |>
  as_tibble() |>
  # mutate(sales = exp(.mean) - 1) |>
  mutate(sales = .mean) |>
  select(-.model, -.mean)


aligned_data <- align_predictions(naive_forecasts, validation)
accuracy_naive <- calculate_metrics(aligned_data)


Seasonal Naive Baseline

In [ ]:
path <- "models/baseline/snaive.rds"

# load fit or fit it and store the fit
if (file.exists(path)) {
  fit <- readRDS(path)
} else {
  fit <- train |>
    model(
      SNAIVE(sales)
    )
  saveRDS(fit, path)
}

snaive_forecasts <- fit |>
  forecast(h = 28) |>
  as_tibble() |>
  # mutate(sales = exp(.mean) - 1) |>
  mutate(sales = .mean) |>
  select(-.model, -.mean)


aligned_data <- align_predictions(snaive_forecasts, validation)
accuracy_snaive <- calculate_metrics(aligned_data)


ARIMA Baseline

In [ ]:
path <- "models/baseline/arima.rds"

# load fit or fit it and store the fit
if (file.exists(path)) {
  fit <- readRDS(path)
} else {
  fit <- train |>
    model(
      ARIMA(sales)
    )
  saveRDS(fit, path)
}

arima_forecasts <- fit |>
  forecast(h = 28) |>
  as_tibble() |>
  # mutate(sales = exp(.mean) - 1) |>
  mutate(sales = .mean) |>
  select(-.model, -.mean)


aligned_data <- align_predictions(arima_forecasts, validation)
accuracy_arima <- calculate_metrics(aligned_data)


ETS Baseline

In [ ]:
path <- "models/baseline/ets.rds"

# load fit or fit it and store the fit
if (file.exists(path)) {
  fit <- readRDS(path)
} else {
  fit <- train |>
    model(
      ETS(sales)
    )
  saveRDS(fit, path)
}

ets_forecasts <- fit |>
  forecast(h = 28) |>
  as_tibble() |>
  # mutate(sales = exp(.mean) - 1) |>
  mutate(sales = .mean) |>
  select(-.model, -.mean)


aligned_data <- align_predictions(ets_forecasts, validation)
accuracy_ets <- calculate_metrics(aligned_data)


Checking performance of ensemble of baseline models

In [ ]:
ensemble_forecasts <- naive_forecasts |>
    inner_join(snaive_forecasts, by = c("product", "day"), suffix = c("_naive", "_snaive")) |>
    inner_join(arima_forecasts, by = c("product", "day")) |>
    inner_join(ets_forecasts, by = c("product", "day"), suffix = c("_arima", "_ets")) |>
    mutate(
        sales = (sales_naive + sales_snaive + sales_arima + sales_ets) / 4
    ) |>
    select(product, day, sales)
aligned_data <- align_predictions(ensemble_forecasts, validation)
accuracy_ensemble <- calculate_metrics(aligned_data)
accuracy_ensemble


Evaluate baselines

In [ ]:
sums_mean <- mean_forecasts |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))

sums_naive <- naive_forecasts |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))

sums_snaive <- snaive_forecasts |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))

sums_arima <- arima_forecasts |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))

sums_ets <- ets_forecasts |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))


sums_val <- validation |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))

# accuracy table
baseline_accuracy <- bind_rows(
  mean = accuracy_mean,
  naive = accuracy_naive,
  snaive = accuracy_snaive,
  arima = accuracy_arima,
  ets = accuracy_ets,
  .id = "model"
)

print(baseline_accuracy)

# plotting against validation to see what is being captured
(sums_val |> autoplot(.vars = sum)) +
  autolayer(sums_naive, sum, color = "red") +
  autolayer(sums_snaive, sum, color = "purple") +
  autolayer(sums_arima, sum, color = "blue") +
  autolayer(sums_ets, sum, color = "orange")


# Actual interesting model (basedline)

Dynamic regression on Sell Price as predictor

In [ ]:
path <- "models/arima_model.rds"

if (file.exists(path)) {
  arima_fit <- readRDS(path)
} else {
  arima_fit <- train |>
    model(
      ARIMA(sales ~ sell_price)
    )

  saveRDS(arima_fit, path)
}


Naive Price

In [ ]:
path <- "models/prices/naive.rds"

# load fit or fit it and store the fit
if (file.exists(path)) {
  price_fit_naive <- readRDS(path)
} else {
  # fitting a naive
  price_fit_naive <- train |>
    model(NAIVE(sell_price))

  saveRDS(price_fit_naive, path)
}

# fitting a naive
price_fit_naive <- train |>
  model(NAIVE(sell_price))

sell_price_future_naive <- price_fit_naive |>
  forecast(h = "28 days") |>
  as_tsibble() |>
  select(product, day, .mean) |>
  rename(sell_price = .mean)

naive_forecasts <- forecast(arima_fit, new_data = sell_price_future_naive) |>
  as_tibble() |>
  mutate(sales = exp(.mean) - 1) |>
  select(day, product, sales)


aligned_data <- align_predictions(naive_forecasts, validation)
accuracy_naive <- calculate_metrics(aligned_data)


Snaive Price

In [ ]:
path <- "models/prices/snaive.rds"

# load fit or fit it and store the fit
if (file.exists(path)) {
  price_fit_snaive <- readRDS(path)
} else {
  # fitting a seasonal naive
  price_fit_snaive <- train |>
    model(SNAIVE(sell_price))

  saveRDS(price_fit_snaive, path)
}

# forecast price
sell_price_future_snaive <- price_fit_snaive |>
  forecast(h = "28 days") |>
  as_tsibble() |>
  select(product, day, .mean) |>
  rename(sell_price = .mean)

# forecast sales based on this price as future data
snaive_forecasts <- forecast(arima_fit, new_data = sell_price_future_snaive) |>
  as_tibble() |>
  mutate(sales = exp(.mean) - 1) |>
  select(day, product, sales)


aligned_data <- align_predictions(snaive_forecasts, validation)
accuracy_snaive <- calculate_metrics(aligned_data)


ARIMA price

In [ ]:
path <- "models/prices/arima.rds"

# load fit or fit it and store the fit
if (file.exists(path)) {
  price_fit_arima <- readRDS(path)
} else {
  # fitting an arima, probably need to decide what kind of d is nice
  price_fit_arima <- train |>
    model(ARIMA(sell_price))

  saveRDS(price_fit_arima, path)
}

sell_price_future_arima <- price_fit_arima |>
  forecast(h = "28 days") |>
  as_tsibble() |>
  select(product, day, .mean) |>
  rename(sell_price = .mean)

arima_forecasts <- forecast(arima_fit, new_data = sell_price_future_arima) |>
  as_tibble() |>
  mutate(sales = exp(.mean) - 1) |>
  select(day, product, sales)


aligned_data <- align_predictions(arima_forecasts, validation)
accuracy_arima <- calculate_metrics(aligned_data)


ETS Price

In [ ]:
path <- "models/prices/ets.rds"

# load fit or fit it and store the fit
if (file.exists(path)) {
  price_fit_ets <- readRDS(path)
} else {
  # fitting an ets
  price_fit_ets <- train |>
    model(ETS(sell_price))

  saveRDS(price_fit_ets, path)
}

sell_price_future_ets <- price_fit_ets |>
  forecast(h = "28 days") |>
  as_tsibble() |>
  select(product, day, .mean) |>
  rename(sell_price = .mean)

ets_forecasts <- forecast(arima_fit, new_data = sell_price_future_ets) |>
  as_tibble() |>
  mutate(sales = exp(.mean) - 1) |>
  select(day, product, sales)


aligned_data <- align_predictions(ets_forecasts, validation)
accuracy_ets <- calculate_metrics(aligned_data)


Evaluate them

In [ ]:
sums_naive <- naive_forecasts |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))

sums_snaive <- snaive_forecasts |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))

sums_arima <- arima_forecasts |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))

sums_ets <- ets_forecasts |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))


sums_val <- validation |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))

# accuracy table
dynamic_price_accuracy <- bind_rows(
  naive = accuracy_naive,
  snaive = accuracy_snaive,
  arima = accuracy_arima,
  ets = accuracy_ets,
  .id = "model"
)

print(dynamic_price_accuracy)

# plotting against validation to see what is being captured
(sums_val |> autoplot(.vars = sum)) +
  autolayer(sums_naive, sum, color = "red") +
  autolayer(sums_snaive, sum, color = "purple") +
  autolayer(sums_arima, sum, color = "blue") +
  autolayer(sums_ets, sum, color = "orange")


Calander predictors incorporated

In [ ]:
# extending it by adding calander data
sell_price_future_naive_cal <- inner_join(
    sell_price_future_naive,
    dates,
    by = c("day")
)

sell_price_future_snaive_cal <- inner_join(
    sell_price_future_snaive,
    dates,
    by = c("day")
)

sell_price_future_arima_cal <- inner_join(
    sell_price_future_arima,
    dates,
    by = c("day")
)

sell_price_future_ets_cal <- inner_join(
    sell_price_future_ets,
    dates,
    by = c("day")
)


In [ ]:
sell_price_future_ets_cal


In [ ]:
View(dates)


arima model with events and date types incorporated

In [ ]:
path <- "models/arima_models/arima_model_cal_rand.rds"

if (file.exists(path)) {
  arima_fit_cal <- readRDS(path)
} else {
  arima_fit_cal <- train |>
    model(
      ARIMA(
        sales ~ sell_price +
          # do you need to specify these?
          # PDQ(0,0,0) + pdq(d = 0)+

          # weekdays (Sunday reference)
          wd_monday + wd_tuesday + wd_wednesday +
          wd_thursday + wd_friday + wd_saturday +

          # months (December reference)
          m_january + m_february + m_march + m_april +
          m_may + m_june + m_july + m_august +
          m_september + m_october + m_november +

          # events
          et_sporting + et_cultural +
          et_national + et_religious +

          snap_TX

        # fourier(period = "week", K = 1) +    # captures weekly patterns
        # fourier(period = "year", K = 5)  # captures yearly patterns
      )
    )

  saveRDS(arima_fit_cal, path)
}


perform the forecasts

In [ ]:
# naive
naive_forecasts <- forecast(arima_fit_cal, new_data = sell_price_future_naive_cal) |>
  as_tibble() |>
  # mutate(sales = exp(.mean) - 1) |>
  mutate(sales = .mean) |>
  select(day, product, sales)


aligned_data <- align_predictions(naive_forecasts, validation)
accuracy_naive <- calculate_metrics(aligned_data)

# snaive
snaive_forecasts <- forecast(arima_fit_cal, new_data = sell_price_future_snaive_cal) |>
  as_tibble() |>
  # mutate(sales = exp(.mean) - 1) |>
  mutate(sales = .mean) |>
  select(day, product, sales)


aligned_data <- align_predictions(snaive_forecasts, validation)
accuracy_snaive <- calculate_metrics(aligned_data)

# arima
arima_forecasts <- forecast(arima_fit_cal, new_data = sell_price_future_arima_cal) |>
  as_tibble() |>
  # mutate(sales = exp(.mean) - 1) |>
  mutate(sales = .mean) |>
  select(day, product, sales)


aligned_data <- align_predictions(arima_forecasts, validation)
accuracy_arima <- calculate_metrics(aligned_data)

# ets
ets_forecasts <- forecast(arima_fit_cal, new_data = sell_price_future_ets_cal) |>
  as_tibble() |>
  # mutate(sales = exp(.mean) - 1) |>
  mutate(sales = .mean) |>
  select(day, product, sales)


aligned_data <- align_predictions(ets_forecasts, validation)
accuracy_ets <- calculate_metrics(aligned_data)


evaluate them

In [ ]:
sums_naive <- naive_forecasts |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))

sums_snaive <- snaive_forecasts |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))

sums_arima <- arima_forecasts |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))

sums_ets <- ets_forecasts |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))


sums_val <- validation |>
  as_tsibble(index = day, key = product) |>
  index_by(day) |>
  summarise(sum = sum(sales, na.rm = TRUE))

# accuracy table
dynamic_price_accuracy <- bind_rows(
  naive = accuracy_naive,
  snaive = accuracy_snaive,
  arima = accuracy_arima,
  ets = accuracy_ets,
  .id = "model"
)

print(dynamic_price_accuracy)

# plotting against validation to see what is being captured
(sums_val |> autoplot(.vars = sum)) +
  autolayer(sums_naive, sum, color = "red") +
  autolayer(sums_snaive, sum, color = "purple") +
  autolayer(sums_arima, sum, color = "blue") +
  autolayer(sums_ets, sum, color = "orange")


In [ ]:
arima_fit_cal


### Here we check the sum of all sales to see if we in general are capturing this or not

In [ ]:
# interesting variables

# price => we can probaby try out
# naive (easy, assumption that prices stay the same)
# ets (makes sense, prices change and most recent observations should weigh more)
# Arima (just to see what pops out)
# snaive?

# events
# probaby don't need to predict it,
# these we can probably look up from the years before

# here we might want to either focus on the category itself or on the event

# snap_TX might be interesting? can probably look it up

# weekday   wday month could be interesting, these we also don't need to predict,
# maybe include feature "week", so what week it is


In [ ]:
# comparing all accuracies
print(baseline_accuracy)
print(dynamic_price_accuracy)


In [ ]:
print(n = 28, sell_price_future_ets |> summarise(last_day = max(day)))


In [ ]:
validation


In [ ]:
ets_forecasts
